# Shipment Tracker API

In [1]:
import pandas as pd
# Set display option to show all columns
pd.set_option('display.max_columns', None)

In [2]:
from functions0_basics import get_files_path, shipments_not_delivered, get_API_details
excel_path, report_path = get_files_path("Albert")
shipments_not_delivered = shipments_not_delivered(excel_path)

from functions1_TNT_requests import make_tnt_requests
tnt_results, len_shipm_numbers = make_tnt_requests(shipments_not_delivered)

from functions2_TNT_dataframe import tnt_to_dataframe
tnt_df = tnt_to_dataframe(tnt_results,len_shipm_numbers)
display(tnt_df.head(3), tnt_df.tail(3))

**150 shipments NOT DELIVERED in your file**

Status,Carrier,In Transit,Exception,Totals
0,DHL,16,0,16
1,NACEX,2,1,3
2,SEUR,1,1,2
3,TNT,127,2,129


Total TNT shipment numbers: 129


,Carrier,Client Reference,Shipment Num.,Origin Date,From (City),From (Country),To (City),To (Country),Num. of Pieces,Processing Days,Summary Code,Signatory,Carrier Code Status,Last Update (Date),Last Update (Hour),Last Location,Last Action,Exception Notification,URL
0,TNT,DSD/140232,663782937,24-11-2023,Barcelona,Spain,Krnjesevci,Serbia,1,22,In Transit,,HW,24-11-2023,21:51,Barcelona,El envío ha llegado a la ubicación de TNT,,https://www.tnt.com/express/en_gc/site/shippin...
1,TNT,DSD/142142,663787660,28-11-2023,Barcelona,Spain,Shkoder,Albania,1,17,Delivered,L HABIBI,OK,15-12-2023,16:30,Albania Others,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...
2,TNT,DSD/146902,663806545,07-12-2023,Barcelona,Spain,Lipari,Italy,1,9,In Transit,,NPD,13-12-2023,14:57,Messina,El destinatario ha solicitado una notificación...,,https://www.tnt.com/express/en_gc/site/shippin...


,Carrier,Client Reference,Shipment Num.,Origin Date,From (City),From (Country),To (City),To (Country),Num. of Pieces,Processing Days,Summary Code,Signatory,Carrier Code Status,Last Update (Date),Last Update (Hour),Last Location,Last Action,Exception Notification,URL
126,TNT,DSD/148038,663811633,11-12-2023,Barcelona,Spain,Ajaccio,France,1,3,Delivered,isulatec,OK,14-12-2023,09:30,Ajaccio,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...
127,TNT,DSD/148044,663811718,11-12-2023,Barcelona,Spain,Hennebont,France,2,3,Delivered,eveno,OK,14-12-2023,10:12,Vannes,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...
128,TNT,DSD/148052,663811704,11-12-2023,Barcelona,Spain,Corignac,France,2,3,Delivered,gillaizeau,OK,14-12-2023,11:17,Angouleme,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...


In [3]:
tnt_df.to_excel("./Track Reports/tnt_clean_report.xlsx")

In [7]:
from functions1_DHL_requests import make_dhl_requests
all_dhl_results, max_dhl_shipm = make_dhl_requests(shipments_not_delivered)

In [8]:
from functions2_DHL_dataframe import dhl_to_dataframe
dhl_df = dhl_to_dataframe(all_dhl_results, shipments_not_delivered, max_dhl_shipm)
dhl_df

,Carrier,Client Reference,Shipment Num.,Service,Origin Date,From (City),From (Country),To (City),To (Country),Num. of Pieces,Processing Days,Carrier Status,Last Update (Date),Last Update (Hour),Last Location (City),Last Location (Country),Last Action,Exception Notification,Shipment URL,POD Status,POD Link,POD Signature Link,Remark,Next Steps,Estimated Date Delivery,Estimated Time Delivery
0,DHL,DSD145944,4537224576,express,07-12-2023,Barcelona,Spain,Bristol,UK,2,9,Delivered,14-12-2023,09:46,Bristol,UK,Delivered,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,,,,
1,DHL,DSD146004,5764174150,express,07-12-2023,Barcelona,Spain,Swindon,UK,4,9,Delivered,15-12-2023,12:47,Swindon,UK,Delivered,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,,,,
2,DHL,DSD146154,5734665823,express,11-12-2023,Barcelona,Spain,Manchester,UK,1,5,Delivered,14-12-2023,10:24,Manchester,UK,Delivered,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,,,,
3,DHL,DSD146616,8939615123,express,11-12-2023,Barcelona,Spain,Manchester,UK,1,5,Delivered,14-12-2023,11:00,Manchester,UK,Delivered,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,,,,
4,DHL,DSD146618,6285974225,express,11-12-2023,Barcelona,Spain,Manchester,UK,2,5,Delivered,14-12-2023,11:56,Manchester,UK,Delivered,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,,,,
5,DHL,DSD146628,8939605964,express,07-12-2023,Barcelona,Spain,Southampton,UK,2,9,Delivered,14-12-2023,10:52,Southampton,UK,Delivered,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,,,,
6,DHL,DSD144508,3764749100,express,11-12-2023,Barcelona,Spain,Birmingham,UK,1,5,In Transit,16-12-2023,03:21,East Midlands,UK,Clearance processing complete at EAST MIDLANDS...,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://proview.dhl.com/proview/adhocnotify?id...,https://proview.dhl.com/proview/adhocnotify?id...,,,,
7,DHL,DSD146556,4884823613,express,11-12-2023,Barcelona,Spain,Southampton,UK,2,5,Delivered,14-12-2023,14:30,Southampton,UK,Delivered,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,,,,
8,DHL,DSD146642,4884783444,express,11-12-2023,Barcelona,Spain,Stansted,UK,2,5,Delivered,14-12-2023,12:53,Stansted,UK,Delivered,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,,,,
9,DHL,DSD146736,3746403511,express,11-12-2023,Barcelona,Spain,Gatwick,UK,2,5,Delivered,14-12-2023,15:56,Gatwick,UK,Delivered,,https://www.dhl.com/es-en/home/tracking/tracki...,False,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,https://webpod.dhl.com/webPOD/DHLePODRequest?h...,,,,


In [9]:
dhl_df.to_excel("./Track Reports/dhl_clean_report.xlsx")